In [21]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import sys
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

# 定义模型

In [22]:
num_hiddens = 256
# rnn_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens) # 已测试
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)

In [23]:
num_steps = 35
batch_size = 2
state = None
X = torch.rand(num_steps, batch_size, vocab_size)
Y, state_new = rnn_layer(X, state)
print(Y.shape, len(state_new), state_new[0].shape)

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [27]:
# 本类已保存在d2lzh_pytorch包中方便以后使用
class RNNModel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super(RNNModel,self).__init__()
        self.rnn=rnn_layer
        self.hidden_size=rnn_layer.hidden_size*(2 if rnn_layer.bidirectional else 1)
        self.vocab_size=vocab_size
        self.dense=nn.Linear(self.hidden_size,vocab_size)
        self.state=None
    def forward(self,inputs,state):# inputs: (batch, seq_len)
        # 获取one-hot向量表示
        X=d2l.to_onehot(inputs,self.vocab_size)
        Y,self.state=self.rnn(torch.stack(X),state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output=self.dense(Y.view(-1,Y.shape[-1]))
        return output,self.state

# 训练模型

In [28]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        if state is not None:
            if isinstance(state, tuple): # LSTM, state:(h, c)  
                state = (state[0].to(device), state[1].to(device))
            else:   
                state = state.to(device)
            
        (Y, state) = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [29]:
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('雪地', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'雪地此相相相相相相相相相'

In [30]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state = (state[0].detach(), state[1].detach())
                else:   
                    state = state.detach()
    
            (output, state) = model(X, state) # output: 形状为(num_steps * batch_size, vocab_size)
            
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            # 梯度裁剪
            d2l.grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [31]:
#使用和上一节实验中一样的超参数（除了学习率）来训练模型。
num_epochs, batch_size, lr, clipping_theta = 500, 10, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 10, 10, ['雪地', '雪地']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 10, perplexity 78.246175, time 0.02 sec
 - 雪地          
 - 雪地          
epoch 20, perplexity 69.790353, time 0.02 sec
 - 雪地          
 - 雪地          
epoch 30, perplexity 65.394586, time 0.00 sec
 - 雪地          
 - 雪地          
epoch 40, perplexity 59.792452, time 0.00 sec
 - 雪地          
 - 雪地          
epoch 50, perplexity 51.236719, time 0.01 sec
 - 雪地里         
 - 雪地里         
epoch 60, perplexity 38.283657, time 0.00 sec
 - 雪地里         
 - 雪地里         
epoch 70, perplexity 21.679526, time 0.01 sec
 - 雪地里    ：你缠与 
 - 雪地里    ：你缠与 
epoch 80, perplexity 11.783931, time 0.00 sec
 - 雪地里   却：我们  
 - 雪地里   却：我们  
epoch 90, perplexity 6.303640, time 0.01 sec
 - 雪地里 爱 他们说零是 
 - 雪地里 爱 他们说零是 
epoch 100, perplexity 3.752505, time 0.00 sec
 - 雪地里 爱 他们说零下已
 - 雪地里 爱 他们说零下已
epoch 110, perplexity 2.517347, time 0.00 sec
 - 雪地里 爱 他们说零下已
 - 雪地里 爱 他们说零下已
epoch 120, perplexity 1.884444, time 0.02 sec
 - 雪地里 爱 他们说零下已
 - 雪地里 爱 他们说零下已
epoch 130, perplexity 1.564456, time 0.00 sec
 - 雪地里 爱 他们说零下已
 - 